In [ ]:
"""
load the HE cell assignment from assignHE.py
"""
# import pandas as pd
# cell_path = '/mnt/data10/shared/yujie/DCIS/ANALYSIS/HE_cellAssignment/exc44_assigned_CL_100_5.csv'

# assigned_cells_df = pd.read_csv(cell_path)  

# assigned_cells_df['clusterId'] = assigned_cells_df['clusterId'].fillna('None')
# none_cluster_counts = assigned_cells_df[assigned_cells_df['clusterId'] == 'None'].groupby('patientNum').size()
# none_cluster_counts_df = none_cluster_counts.reset_index(name='None_Count')

# total_counts = assigned_cells_df.groupby('patientNum').size()
# total_counts_df =total_counts.reset_index(name='total_Count')

# counts_df = pd.merge(none_cluster_counts_df, total_counts_df, on='patientNum')
# counts_df['None_percentage'] = counts_df.apply(
#     lambda row: row['None_Count'] / row['total_Count'],
#     axis=1
# )

# # Display the result
# print(counts_df)

# #NA means the HE cell does NOT belong to any IHC-community


'''
stats for number of ducts
'''
import pandas as pd
# cell_path = '/mnt/data10/shared/yujie/DCIS/ANALYSIS/HE_cellAssignment/exc44_assigned_CL_100_5.csv'

# assigned_cells_df = pd.read_csv(cell_path)
# assigned_cells_df.groupby(['slideId','ductId']).count()

cell_path = '/mnt/data10/shared/yujie/DCIS/ANALYSIS/HE_cellAssignment/biopsy_assigned_CL_100_5.csv'

assigned_cells_df = pd.read_csv(cell_path)
assigned_cells_df.groupby(['slideId','ductId']).count()

In [ ]:
import os 
import json
import pickle
from scipy.stats import kurtosis,skew
import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt
########################################################################################
#load morphology features and texture features for each HE cell then aggregate at ROI level
def AggreCellFeatures(text_folder,morph_folder,assigned_cells_df ,out_path):
    
    cells_df = assigned_cells_df.dropna(axis=0)#drop unassigned cells
    print(f'original count:{len(assigned_cells_df)} ,no-NA count(HE cells within neighborhood):{len(cells_df)}')

    df_list=[]
    for slideId in cells_df['slideId'].unique():
        print(f'{slideId} starts processing ...')
        if not os.path.exists(os.path.join(morph_folder,f'{slideId}.json')):
            print(f'{slideId}.json does not exist')
            continue
        with open(os.path.join(morph_folder,f'{slideId}.json'),'r') as f:
            morph_feat=json.load(f)
        with open(os.path.join(text_folder,f'{slideId}.pkl'),'rb') as f:
            text_feat=pickle.load(f)
        cell_slide = cells_df[cells_df['slideId']==slideId]
        morph_df=pd.DataFrame(morph_feat)
        text_df = pd.DataFrame(text_feat)
        merge1_df = pd.merge(cell_slide, morph_df, left_on='Object ID', right_on='cellId', how='left')
        slide_df = pd.merge(merge1_df, text_df, on='cellId', how='left')
        df_list.append(slide_df)
    concat_df = pd.concat(df_list)
    final_df=concat_df.drop(columns=['CurvMean','CurvStd','CurvMax','CurvMin']).dropna(axis=0)

    #print(final_df.columns)
    excluded_columns = ['Object ID', 'patientNum', 'slideId', 'ductId', 'layer', 'clusterId', 'X', 'Y', 'cellId']
    included_columns = final_df.columns.difference(excluded_columns)

    def kurtosis_func(x):
        return kurtosis(x, fisher=True)

    def skewness_func(x):
        return skew(x)
    agg_dict = {col: ['mean', 'std', 'max', 'min', kurtosis_func, skewness_func] for col in included_columns}
    flat_columns = []
    for main_col, sub_cols in agg_dict.items():
        for op in sub_cols:
            if callable(op):  # For custom functions, use the function's name
                op_name = op.__name__.replace('_func', '')
            else:
                op_name = op
            flat_columns.append(f'{main_col}_{op_name}')
    ####################aggregate at community level
    comm_agg = final_df.groupby(['slideId', 'clusterId'], as_index=False).agg(agg_dict)
    comm_agg.columns = ['slideId', 'clusterId'] + flat_columns
    #print(comm_agg.head())
    #remove rows with NAs
    comm_agg=comm_agg.dropna()
    if out_path is not None:
        comm_agg.to_csv(out_path,index=False)
    return comm_agg
########################################################################################
#Concat spatial function values and ihc counts from R (merge column-wise) 
#(row number of cell_aggre is far less than spat )[BECAUSE SEGMENTATION QUALITY, DUCTS IN HE AER UNDER_SEGMENTED]
def concatCellSpat(comm_agg,spat,counts,id_df,out_path):
    spat['patientNum'] = spat['patient_cluster'].apply(lambda x: x.split('_')[0])
    spat['clusterId'] = spat['patient_cluster'].apply(lambda x: x.split('_')[1])
    spat=pd.merge(spat,id_df[['patientNum','slideId']],on='patientNum')
    spat = spat.drop(columns=['patientNum','patient_cluster'])

    counts=counts.dropna(axis=1,how='all')
    counts['patientNum'] = counts['patient_cluster'].apply(lambda x: x.split('_')[0])
    counts['clusterId'] = counts['patient_cluster'].apply(lambda x: x.split('_')[1])
    counts=pd.merge(counts,id_df[['patientNum','slideId']],on='patientNum')
    counts = counts.drop(columns=['patientNum','patient_cluster','count_Glut1'])###EXCLUDE Glut1 

    comm_agg['clusterId'] = comm_agg['clusterId'].astype(str)
    merge1=pd.merge(comm_agg,spat,on=['slideId','clusterId'])
    merged_df = pd.merge(merge1,counts,on=['slideId','clusterId'])

    ##Remove columns with any NA(usually its 0 or a lot)
    final_df = merged_df.dropna(axis=1,how='any')
    print(f'merged_df col no. {len(merged_df.columns)},final_df column no. {len(final_df.columns)}')
    final_df.to_csv(out_path,index=False)
    
    return final_df
##################################################
#####PREPARE TRAINING/VALIDATION SPLIT
def prepareSplit(final_df,id_df,json_path):
    # Get the upstage Upstage
    final_df = pd.merge(final_df, id_df[['slideId','Upstage']], on = "slideId")
    #print(final_df['Upstage'].value_counts())

    experiment_indexes = {}
    positive_slideIds = final_df[final_df['Upstage'] == 1]['slideId'].unique()
    for valid_slideId in positive_slideIds:
        validation_set_positives = final_df[final_df['slideId'] == valid_slideId]
        remaining_data = final_df[~(final_df['slideId'] == valid_slideId)]
        experiment_indexes[int(valid_slideId)] = {}
        for i in range(3):
            np.random.seed(i)

            sampled_validation_negatives_indexes = remaining_data[remaining_data['Upstage'] == 0].sample(n=len(validation_set_positives)).index
            training_positives_indexes = remaining_data[remaining_data['Upstage'] == 1].index
            if len(training_positives_indexes) > len(remaining_data[remaining_data['Upstage'] == 0]):
                sampled_training_negatives_indexes = remaining_data[remaining_data['Upstage'] == 0].sample(n=len(training_positives_indexes),
                                                                                                          replace=True).index
            else:
                sampled_training_negatives_indexes = remaining_data[remaining_data['Upstage'] == 0].sample(n=len(training_positives_indexes)).index

            experiment_indexes[valid_slideId][i]={
                'validation_indexes': validation_set_positives.index.union(sampled_validation_negatives_indexes).tolist(),
                'training_indexes': training_positives_indexes.union(sampled_training_negatives_indexes).tolist()
            }
    
    with open(json_path,'w') as f:
        json.dump(experiment_indexes,f,indent=4)
    return experiment_indexes
########################################################################################
#Plot the ROI count for each slide(upstage vs indolent)
def plotROIdist(final_df,id_df):

    final_df['slideId'] = final_df['slideId'].astype(str)
    id_df['slideId'] = id_df['slideId'].astype(str)
    merged_df = pd.merge(final_df, id_df, on='slideId')

    count_per_slideId = merged_df.groupby(['slideId', 'Upstage']).size().reset_index(name='counts')

    indolent_df = count_per_slideId[count_per_slideId['Upstage'] == 0]
    upstage_df = count_per_slideId[count_per_slideId['Upstage'] > 0]

    indolent_df = indolent_df.sort_values(by='counts', ascending=True)
    upstage_df = upstage_df.sort_values(by='counts', ascending=True)

    # Plotting
    positions_a = np.arange(len(indolent_df))
    positions_b = np.arange(len(indolent_df), len(indolent_df) + len(upstage_df))

    plt.figure(figsize=(12, 8))
    plt.bar(positions_a, indolent_df['counts'], label='Indolent')
    plt.bar(positions_b, upstage_df['counts'], label='Upstage', color='orange')
    plt.axvline(x=max(positions_a), color='r', linestyle='--')
    plt.xticks(np.concatenate([positions_a, positions_b]), np.concatenate([indolent_df['slideId'], upstage_df['slideId']]), rotation=45, ha='right')

    plt.xlabel('Slide ID')
    plt.ylabel('Count of Samples')
    plt.title('Sample Counts per Slide ID by Upstage Status')
    plt.legend()
    plt.tight_layout()
    plt.show()

    print(f'Total number of samples Indolent: {indolent_df["counts"].sum()}')
    print(f'Total number of samples Upstage: {upstage_df["counts"].sum()}')


##### WHOLE DUCT

In [ ]:
##CA9&LAMP2b 100 5

stage = 'biopsy'
id_df = pd.read_csv('/mnt/data10/shared/yujie/DCIS/patientNum_slideId_upstage_biopsy.csv')
text_folder = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/{stage}_HE_cell_features/texture'
morph_folder = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/{stage}_HE_cell_features/morphology'
assignedCell_path = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/HE_cellAssignment/{stage}_assigned_CL_100_5.csv'

#from process_spatialFunc.ipynb
spat=pd.read_csv(f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/R_analysis/{stage}_CL_100_5/spat_integration.csv')
counts = pd.read_csv(f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/R_analysis/{stage}_CL_100_5/ihc_counts.csv')

result_dir = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/CL_100_5_result/{stage}'
os.makedirs(result_dir, exist_ok=True)
comm_agg_path = os.path.join(result_dir,'commAgg_morph_text.csv')
final_df_path = os.path.join(result_dir,'allFeatures.csv')
json_path = os.path.join(result_dir,'split_index.json')

####################RUN
assigned_cells_df = pd.read_csv(assignedCell_path) 

comm_agg=AggreCellFeatures(text_folder,morph_folder,assigned_cells_df ,comm_agg_path)
print('AGGREGATE CELL FEATURES DONE')
final_df = concatCellSpat(comm_agg,spat,counts,id_df,final_df_path)
print('MERGE CELL FEATURES WITH SPATIAL FEATURES DONE')

plotROIdist(final_df,id_df)

###########################################################################RUN#########################
stage = 'exc44'
id_df = pd.read_csv('/mnt/data10/shared/yujie/DCIS/patientNum_slideId_upstage_excision.csv')
text_folder = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/{stage}_HE_cell_features/texture'
morph_folder = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/{stage}_HE_cell_features/morphology'
assignedCell_path = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/HE_cellAssignment/{stage}_assigned_CL_100_5.csv'

#from process_spatialFunc.ipynb
spat=pd.read_csv(f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/R_analysis/{stage}_CL_100_5/spat_integration.csv')
counts = pd.read_csv(f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/R_analysis/{stage}_CL_100_5/ihc_counts.csv')


result_dir = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/CL_100_5_result/{stage}'
os.makedirs(result_dir, exist_ok=True)
comm_agg_path = os.path.join(result_dir,'commAgg_morph_text.csv')
final_df_path = os.path.join(result_dir,'allFeatures.csv')
json_path = os.path.join(result_dir,'split_index.json')

#########################RUN
assigned_cells_df = pd.read_csv(assignedCell_path) 

comm_agg=AggreCellFeatures(text_folder,morph_folder,assigned_cells_df ,comm_agg_path)
print('AGGREGATE CELL FEATURES DONE')
final_df = concatCellSpat(comm_agg,spat,counts,id_df,final_df_path)
print('MERGE CELL FEATURES WITH SPATIAL FEATURES DONE')
experiment_indexes = prepareSplit(final_df,id_df,json_path)
print('GENERATE TRAIN/VALID SPLIT DONE')
comm_agg = pd.read_csv(comm_agg_path)
final_df = pd.read_csv(final_df_path)

plotROIdist(final_df,id_df)





In [ ]:
##CA9 100 5

stage = 'biopsy'
id_df = pd.read_csv('/mnt/data10/shared/yujie/DCIS/patientNum_slideId_upstage_biopsy.csv')
text_folder = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/{stage}_HE_cell_features/texture'
morph_folder = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/{stage}_HE_cell_features/morphology'
assignedCell_path = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/HE_cellAssignment/{stage}_assigned_CA9_100_5.csv'

#from process_spatialFunc.ipynb
spat=pd.read_csv(f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/R_analysis/{stage}_C_100_5/spat_integration.csv')
counts = pd.read_csv(f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/R_analysis/{stage}_C_100_5/ihc_counts.csv')

result_dir = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/CA9_100_5_result/{stage}'
os.makedirs(result_dir, exist_ok=True)
comm_agg_path = os.path.join(result_dir,'commAgg_morph_text.csv')
final_df_path = os.path.join(result_dir,'allFeatures.csv')
json_path = os.path.join(result_dir,'split_index.json')

####################RUN
assigned_cells_df = pd.read_csv(assignedCell_path) 

comm_agg=AggreCellFeatures(text_folder,morph_folder,assigned_cells_df ,comm_agg_path)
print('AGGREGATE CELL FEATURES DONE')
final_df = concatCellSpat(comm_agg,spat,counts,id_df,final_df_path)
print('MERGE CELL FEATURES WITH SPATIAL FEATURES DONE')

plotROIdist(final_df,id_df)

###########################################################################RUN#########################
stage = 'exc44'
id_df = pd.read_csv('/mnt/data10/shared/yujie/DCIS/patientNum_slideId_upstage_excision.csv')
text_folder = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/{stage}_HE_cell_features/texture'
morph_folder = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/{stage}_HE_cell_features/morphology'
assignedCell_path = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/HE_cellAssignment/{stage}_assigned_CA9_100_5.csv'

#from process_spatialFunc.ipynb
spat=pd.read_csv(f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/R_analysis/{stage}_C_100_5/spat_integration.csv')
counts = pd.read_csv(f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/R_analysis/{stage}_C_100_5/ihc_counts.csv')


result_dir = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/CA9_100_5_result/{stage}'
os.makedirs(result_dir, exist_ok=True)
comm_agg_path = os.path.join(result_dir,'commAgg_morph_text.csv')
final_df_path = os.path.join(result_dir,'allFeatures.csv')
json_path = os.path.join(result_dir,'split_index.json')

#########################RUN
assigned_cells_df = pd.read_csv(assignedCell_path) 

comm_agg=AggreCellFeatures(text_folder,morph_folder,assigned_cells_df ,comm_agg_path)
print('AGGREGATE CELL FEATURES DONE')
final_df = concatCellSpat(comm_agg,spat,counts,id_df,final_df_path)
print('MERGE CELL FEATURES WITH SPATIAL FEATURES DONE')
experiment_indexes = prepareSplit(final_df,id_df,json_path)
print('GENERATE TRAIN/VALID SPLIT DONE')
comm_agg = pd.read_csv(comm_agg_path)
final_df = pd.read_csv(final_df_path)

plotROIdist(final_df,id_df)

## handle the difference in feature numbers between exc and biopsy
# import pandas as pd
# final_exc = pd.read_csv('/mnt/data10/shared/yujie/DCIS/ANALYSIS/CA9_100_5_result/exc44/allFeatures.csv')
# final_biopsy = pd.read_csv('/mnt/data10/shared/yujie/DCIS/ANALYSIS/CA9_100_5_result/biopsy/allFeatures.csv')
# final_exc_sub = final_exc[final_biopsy.columns]
# final_exc_sub.to_csv('/mnt/data10/shared/yujie/DCIS/ANALYSIS/CA9_100_5_result/exc44/allFeatures.csv',index=False)


In [ ]:
##LAMP2b 100 5

stage = 'biopsy'
id_df = pd.read_csv('/mnt/data10/shared/yujie/DCIS/patientNum_slideId_upstage_biopsy.csv')
text_folder = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/{stage}_HE_cell_features/texture'
morph_folder = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/{stage}_HE_cell_features/morphology'
assignedCell_path = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/HE_cellAssignment/{stage}_assigned_LAMP2b_100_5.csv'

#from process_spatialFunc.ipynb
spat=pd.read_csv(f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/R_analysis/{stage}_L_100_5/spat_integration.csv')
counts = pd.read_csv(f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/R_analysis/{stage}_L_100_5/ihc_counts.csv')

result_dir = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/LAMP2b_100_5_result/{stage}'
os.makedirs(result_dir, exist_ok=True)
comm_agg_path = os.path.join(result_dir,'commAgg_morph_text.csv')
final_df_path = os.path.join(result_dir,'allFeatures.csv')
json_path = os.path.join(result_dir,'split_index.json')

####################RUN
assigned_cells_df = pd.read_csv(assignedCell_path) 

comm_agg=AggreCellFeatures(text_folder,morph_folder,assigned_cells_df ,comm_agg_path)
print('AGGREGATE CELL FEATURES DONE')
final_df = concatCellSpat(comm_agg,spat,counts,id_df,final_df_path)
print('MERGE CELL FEATURES WITH SPATIAL FEATURES DONE')

plotROIdist(final_df,id_df)

###########################################################################RUN#########################
stage = 'exc44'
id_df = pd.read_csv('/mnt/data10/shared/yujie/DCIS/patientNum_slideId_upstage_excision.csv')
text_folder = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/{stage}_HE_cell_features/texture'
morph_folder = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/{stage}_HE_cell_features/morphology'
assignedCell_path = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/HE_cellAssignment/{stage}_assigned_LAMP2b_100_5.csv'

#from process_spatialFunc.ipynb
spat=pd.read_csv(f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/R_analysis/{stage}_L_100_5/spat_integration.csv')
counts = pd.read_csv(f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/R_analysis/{stage}_L_100_5/ihc_counts.csv')


result_dir = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/LAMP2b_100_5_result/{stage}'
os.makedirs(result_dir, exist_ok=True)
comm_agg_path = os.path.join(result_dir,'commAgg_morph_text.csv')
final_df_path = os.path.join(result_dir,'allFeatures.csv')
json_path = os.path.join(result_dir,'split_index.json')

#########################RUN
assigned_cells_df = pd.read_csv(assignedCell_path) 

comm_agg=AggreCellFeatures(text_folder,morph_folder,assigned_cells_df ,comm_agg_path)
print('AGGREGATE CELL FEATURES DONE')
final_df = concatCellSpat(comm_agg,spat,counts,id_df,final_df_path)
print('MERGE CELL FEATURES WITH SPATIAL FEATURES DONE')
experiment_indexes = prepareSplit(final_df,id_df,json_path)
print('GENERATE TRAIN/VALID SPLIT DONE')
comm_agg = pd.read_csv(comm_agg_path)
final_df = pd.read_csv(final_df_path)

plotROIdist(final_df,id_df)





In [ ]:
########simple cross-validation
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
import json
import os
def perform_fold_cv(final_df, id_df, json_path,n=5):
    final_df = pd.merge(final_df, id_df[['slideId', 'Upstage']], on="slideId")
    
    X = final_df.drop(columns=['Upstage'])
    y = final_df['Upstage']
    
    skf = StratifiedKFold(n_splits=n, shuffle=True, random_state=42)
    
    fold_indexes = {}
    fold_indexes[f'{n}_fold'] = {}
    for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
        fold_indexes[f'{n}_fold'][fold] = {
            'training_indexes': train_index.tolist(),
            'validation_indexes': test_index.tolist()
        }
    
    # Save to JSON
    with open(json_path, 'w') as f:
        json.dump(fold_indexes, f, indent=4)
    
    return fold_indexes

# Assuming final_df and id_df are defined, and you have a valid json_path
# fold_indexes = perform_5_fold_cv(final_df, id_df, 'path_to_your_file.json')

stage = 'exc44'
id_df = pd.read_csv('/mnt/data10/shared/yujie/DCIS/patientNum_slideId_upstage_excision.csv')
result_dir = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/CL_100_5_result/{stage}'

final_df_path = os.path.join(result_dir,'allFeatures.csv')
json_path = os.path.join(result_dir,'5fold_split_index.json')

final_df = pd.read_csv(final_df_path)

fold_indexes = perform_fold_cv(final_df, id_df, json_path)


In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt


def read_loss(result_dir,plot=True):
    total_loss_dic = {}
    for i in range(2,11):
        total_loss_dic[f'latent{i}'] = {'mean':None,'std':None}
        loss_list = []
        for j in range(3,11):
            loss_csv = os.path.join(result_dir,f'latent{i}_cluster{j}','min_losses.csv')
            csv_df = pd.read_csv(loss_csv)
            loss_list.append(pd.read_csv(loss_csv)['min_loss'].apply(lambda x:float(x[7:13])).mean())
        total_loss_dic[f'latent{i}']['mean'] = np.mean(np.array(loss_list))
        total_loss_dic[f'latent{i}']['std'] = np.std(np.array(loss_list))
    dist_loss_dic = {}
    for i in range(2,11):
        dist_loss_dic[f'latent{i}'] = {'mean':None,'std':None}
        loss_list = []
        for j in range(3,11):
            loss_csv = os.path.join(result_dir,f'latent{i}_cluster{j}','min_losses.csv')
            csv_df = pd.read_csv(loss_csv)
            loss_list.append(pd.read_csv(loss_csv)['min_dist_loss'].mean())
        dist_loss_dic[f'latent{i}']['mean'] = np.mean(np.array(loss_list))
        dist_loss_dic[f'latent{i}']['std'] = np.std(np.array(loss_list))
    recons_loss_dic = {}
    for i in range(2,11):
        recons_loss_dic[f'latent{i}'] = {'mean':None,'std':None}
        loss_list = []
        for j in range(3,11):
            loss_csv = os.path.join(result_dir,f'latent{i}_cluster{j}','min_losses.csv')
            csv_df = pd.read_csv(loss_csv)
            recons_loss = pd.read_csv(loss_csv)['min_loss'].apply(lambda x:float(x[7:13])).mean()-pd.read_csv(loss_csv)['min_dist_loss'].mean()
            loss_list.append(recons_loss)
        recons_loss_dic[f'latent{i}']['mean'] = np.mean(np.array(loss_list))
        recons_loss_dic[f'latent{i}']['std'] = np.std(np.array(loss_list))


    latent_i_values = range(2, 11)
    mean_total = [total_loss_dic[f'latent{i}']['mean'] for i in latent_i_values]
    mean_dist = [dist_loss_dic[f'latent{i}']['mean'] for i in latent_i_values]
    mean_recons = [recons_loss_dic[f'latent{i}']['mean'] for i in latent_i_values]

    if plot:
        plt.plot(latent_i_values, mean_total, marker='o',label='total')
        plt.plot(latent_i_values, mean_dist, marker='o',label='kmeans')
        plt.plot(latent_i_values, mean_recons, marker='o',label='reconstruction')

        plt.xlabel('Latent i')
        plt.ylabel('Mean Loss')
        plt.legend()
        plt.grid(True)
        plt.show()
    return np.mean(np.array(mean_total)) 

mean_total_list = []
for beta in range(1,11):
    result_dir = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/CL_100_5_result_new/exc44/beta0.{beta}'
    if beta == 10:
        result_dir = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/CL_100_5_result_new/exc44/beta1'
    
    mean_total_loss = read_loss(result_dir,True)
    mean_total_list.append(mean_total_loss)

beta_values = np.arange(0.1,1.1,0.1)
plt.plot(beta_values, mean_total_list, marker='o',label='total')

plt.xlabel('beta')
plt.ylabel('Mean Total Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def read_loss_violin(result_dir, plot=True):
    total_loss_dic = {}
    min_kmeans_loss_clusters = {}
    
    for i in range(2, 11):
        total_loss_dic[f'latent{i}'] = {'loss_list': None, 'mean': None, 'std': None}
        loss_list = []
        kmeans_loss_list = []
        cluster_list = list(range(3, 11))
        
        for j in cluster_list:
            loss_csv = os.path.join(result_dir, f'latent{i}_cluster{j}', 'min_losses.csv')
            csv_df = pd.read_csv(loss_csv)
            loss = pd.read_csv(loss_csv)['min_dist_loss'].mean()
            loss_list.append(loss)
            kmeans_loss = pd.read_csv(loss_csv)['min_dist_loss'].mean()
            kmeans_loss_list.append(kmeans_loss)
            
        total_loss_dic[f'latent{i}']['loss_list'] = loss_list
        total_loss_dic[f'latent{i}']['mean'] = np.mean(np.array(loss_list))
        total_loss_dic[f'latent{i}']['std'] = np.std(np.array(loss_list))
        min_kmeans_loss = min(kmeans_loss_list)
        min_kmeans_loss_index = kmeans_loss_list.index(min_kmeans_loss)
        min_kmeans_loss_clusters[f'latent{i}'] = cluster_list[min_kmeans_loss_index]

    latent_i_values = range(2, 11)
    mean_total = [total_loss_dic[f'latent{i}']['mean'] for i in latent_i_values]
    
    if plot:
        fig, ax = plt.subplots()
        sns.violinplot(data=[loss_list['loss_list'] for _, loss_list in total_loss_dic.items()], ax=ax)
        
        for i, (latent, loss_list) in enumerate(total_loss_dic.items()):
            min_loss = min(loss_list['loss_list'])
            min_loss_index = loss_list['loss_list'].index(min_loss)
            min_loss_cluster = min_kmeans_loss_clusters[latent]
            ax.scatter([i], [min_loss], color='red', zorder=5)
            ax.annotate(f'Cluster: {min_loss_cluster}', xy=(i, min_loss), xytext=(i + 0.2, min_loss - 0.02),
                        arrowprops=dict(facecolor='black', arrowstyle='->'), fontsize=8)
            
        ax.set_xticklabels([f'{i}' for i in latent_i_values])
        ax.set_xlabel('Latent i')
        ax.set_ylabel('Loss')
        ax.set_title('Violin Plot of Kmeans jLoss for Different Latent i')
        plt.grid(True)
        plt.show()
        
    return np.mean(np.array(mean_total))

mean_total_list = []
for beta in range(1,11):
    result_dir = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/CL_100_5_result_new/exc44/beta0.{beta}'
    if beta == 10:
        result_dir = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/CL_100_5_result_new/exc44/beta1'
    
    mean_total_loss = read_loss_violin(result_dir,True)
    mean_total_list.append(mean_total_loss)

beta_values = np.arange(0.1,1.1,0.1)
plt.plot(beta_values, mean_total_list, marker='o',label='total')

plt.xlabel('beta')
plt.ylabel('Mean Total Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
for beta in range(1,11):
    result_dir = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/CL_100_5_result_new/exc44/beta0.{beta}'
    if beta == 10:
        result_dir = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/CL_100_5_result_new/exc44/beta1'
    dist_loss_dic = {}

    i = 7
    loss_list = []
    for j in range(3,11):
        loss_csv = os.path.join(result_dir,f'latent{i}_cluster{j}','min_losses.csv')
        csv_df = pd.read_csv(loss_csv)
        loss_list.append(pd.read_csv(loss_csv)['min_loss'].apply(lambda x:float(x[7:13])).mean())\
        
    cluster_values = range(3,11)
    plt.plot(cluster_values, loss_list, marker='o',label='total')

    plt.xlabel('cluster')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
beta = 5
result_dir = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/CL_100_5_result_new/exc44/train_all/beta0.{beta}'
if beta == 10:
    result_dir = f'/mnt/data10/shared/yujie/DCIS/ANALYSIS/CL_100_5_result_new/exc44/beta1'
dist_loss_dic = {}

i = 7
total_loss_list = []
dist_loss_list = []
rec_loss_list=[]
for j in range(3,11):
    loss_csv = os.path.join(result_dir,f'latent{i}_cluster{j}','min_losses.csv')
    csv_df = pd.read_csv(loss_csv)
    total_loss_list.append(pd.read_csv(loss_csv)['min_loss'].values)
    dist_loss_list.append(pd.read_csv(loss_csv)['min_dist_loss'].values)
    rec_loss_list.append(pd.read_csv(loss_csv)['min_rec_loss'].values)
cluster_values = range(3,11)

plt.plot(cluster_values, total_loss_list, marker='o',label='total')
plt.plot(cluster_values, dist_loss_list, marker='o',label='Kmeans')
plt.plot(cluster_values, rec_loss_list, marker='o',label='reconstruction')
plt.xlabel('cluster')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()